In [1]:
from scipy.stats import norm
from scipy.linalg import cho_factor, cho_solve, cholesky, sqrtm, det
from scipy.linalg import norm as la_norm
import numpy as np

from gproc.generative import sample_at_x
from gproc.kernels import squared_exponential

In [2]:
N = 50

# x = np.random.uniform(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
x = np.linspace(-1, 1, N).reshape(-1, 1) # Reshape to N x 1 matrix
y, prob_y, f = sample_at_x(x, kernel_params={'lengthscale': 0.1, 'variance': 1.4})

model_lengthscale = 0.1
gram = squared_exponential(x, x, lengthscale = model_lengthscale)

In [13]:
def expectation_propagation_probit(observed_y, gram, max_iterations=100, tol=1e-5):
    """
    Computes the expectation propagation (Algorithm 3.5) approximation to the latent
    function implied by the model:

        p(y_i | f_i) = norm_cdf(y_i * f_i)
        p(f | gram) = normal(0, gram)

    We target the posterior:

        p(f | y) = Z * p(y | f) * p(f | gram)

    With an approximation q(f) = normal(mu, Sig).

    :param y: num_observations x 1 numpy array containing 1 or -1
    :param gram: num_observations x num_observations numpy array
    """
    
    # Data size and gram matrix relabelling
    N = y.shape[0]
    K = np.copy(gram)
    
    # Initialise parameter arrays
    nu_site = np.zeros(N)
    tau_site = np.zeros(N)
    nu_cav = np.zeros(N)
    tau_cav = np.zeros(N)
    z = np.zeros(N)
    Sig = np.copy(gram)
    mu_proposed = np.zeros(N)
    
    def cavity_params(Sig, tau_site, nu_site, mu, i):
        """Compute approximate cavity parameters"""
        tau_cav_i = Sig[i, i]**(-1) - tau_site[i]
        nu_cav_i = Sig[i, i]**(-1) * mu[i] - nu_site[i]
        return tau_cav_i, nu_cav_i
    
    def marginal_moments(nu_cav, tau_cav, y, i):
        """Compute marginal moments"""
        z_i = ( y[i] * nu_cav[i] / tau_cav[i] ) / np.sqrt( 1 + tau_cav[i]**(-1) )
        mu_hat = ( nu_cav[i] / tau_cav[i] ) + ( ( y[i] * tau_cav[i]**(-1) * norm.pdf(z_i) ) / ( norm.cdf(z_i) * np.sqrt( 1 + tau_cav[i]**(-1) ) ) )
        var_hat = tau_cav[i]**(-1)  - ( ( tau_cav[i]**(-2) * norm.pdf(z_i) ) / ( ( 1 + tau_cav[i]**(-1) ) * norm.cdf(z_i) ) ) * ( z_i + ( norm.pdf(z_i) / norm.cdf(z_i) ) ) 
        return mu_hat, var_hat
    
    def site_params(var_hat, mu_hat, tau_cav, tau_site, i):
        """Update site parameters"""
        tau_delta = var_hat**(-1) - tau_cav[i] - tau_site[i]
        tau_site_i = tau_site[i] + tau_delta
        nu_site_i = var_hat**(-1) * mu_hat - nu_cav[i]
        return tau_delta, tau_site_i, nu_site_i
    
    def posterior_params(Sig, tau_delta, nu_site, i):
        """Update approximate posterior parameters"""
        Sig = Sig - ( ( tau_delta**(-1) + Sig[i, i] )**(-1) * np.outer(Sig[:, i], Sig[:, i]) )
        mu = Sig.dot(nu_site)
        return Sig, mu
    
    def re_posterior_params(tau_site, K):
        """Recompute approximate posterior parameters"""
        S_site_sqrt = np.diag(np.sqrt(tau_site))
        L = cholesky(np.eye(N) + S_site_sqrt.dot(K).dot(S_site_sqrt), lower=True, check_finite=True)
        V = np.linalg.solve(L.T, S_site_sqrt.dot(K))
        Sig = K - np.matmul(V.T, V)
        mu_proposed = Sig.dot(nu_site)
        return L, Sig, mu_proposed
    
    def mark_lik_ev(tau_cav, L, tau_site, nu_site, nu_cav):
        """Compute marginal likelihood log evidence"""        
        one = np.sum( np.log( np.diagonal(L) ) )
        two = 0.5 * nu_site.dot( Sig - np.diag( (tau_cav + tau_site)**(-1) ) ).dot(nu_site)
        three = np.sum( norm.logcdf( ( y * nu_cav / tau_cav ) / ( np.sqrt( 1 + tau_cav**(-1) ) ) ) )
        four = 0.5 * np.sum ( np.log( 1 + tau_site * tau_cav**(-1) ) ) 
        five = 0.5 * (nu_cav / tau_cav).dot( np.diag(tau_cav) ).dot( np.diag( (tau_cav + tau_site)**(-1) ) ).dot( np.diag(tau_site).dot(nu_cav / tau_cav) - 2 * nu_site)
        return one + two + three + four + five
    
    for k in range(1, max_iterations):
        mu = mu_proposed
        
        for i in range(N):
            # Compute approximate cavity parameters
            tau_cav[i], nu_cav[i] = cavity_params(Sig, tau_site, nu_site, mu, i)
            
            # Compute marginal moments
            mu_hat, var_hat = marginal_moments(nu_cav, tau_cav, y, i)

            # Update site parameters
            tau_delta, tau_site[i], nu_site[i] = site_params(var_hat, mu_hat, tau_cav, tau_site, i)
            
            # Update approximate posterior parameters
            Sig, mu = posterior_params(Sig, tau_delta, nu_site, i)
            
        print(nu_cav / tau_cav )
        # Recompute the approximate posterior parameters        
        L, Sig, mu_proposed = re_posterior_params(tau_site, K)

        # Check convergence condition
        if la_norm(mu_proposed - mu) < tol:
            converged = True
            break
    
    # Compute marginal likelihood log evidence
    log_evidence = mark_lik_ev(tau_cav, L, tau_site, nu_site, nu_cav, tau_cav)
    
    return nu_site, tau_site, log_evidence, converged
        
    

In [14]:
expectation_propagation_probit(y, gram)

[0.         0.55950948 0.83390862 0.99810241 1.10545143 1.17860532
 1.22915097 1.26374156 1.28651308 1.30023358 0.55745323 0.03395932
 0.16190546 0.29398668 0.43116564 0.57104597 0.70931447 0.84100399
 0.96147719 1.06706712 1.15539782 1.22545796 1.27750437 1.31285585
 1.33362199 1.3424044  1.34200417 1.33516462 1.324371   1.31171742
 1.29884071 1.28691166 1.27666978 1.26848603 1.2624398  1.25839906
 1.25609552 1.25518967 1.25532278 1.25615501 1.25739022 1.25878924
 1.2601737  1.26142313 1.26246762 1.26327807 1.26385584 1.26422292
 1.2644133  1.26446608]
[ 2.08295937  2.15998552  2.15821288  2.06951962  1.89213595  1.63807663
  1.3353441   1.01929567  0.71844895  0.83848409  0.43452027 -0.03078569
 -0.18753468 -0.29286317 -0.34787669 -0.35354788 -0.3110976  -0.2228578
 -0.09329721  0.07028378  0.25792212  0.45816019  0.65973517  0.85303528
  1.0308405   1.18834735  1.32276006  1.43273982  1.51789087  1.57835941
  1.6145658   1.62706809  1.61654525  1.58388     1.53030997  1.45760006
  1

/tmp/ipykernel_69560/1700823849.py:40: RuntimeWarning: invalid value encountered in sqrt
  z_i = ( y[i] * nu_cav[i] / tau_cav[i] ) / np.sqrt( 1 + tau_cav[i]**(-1) )
/tmp/ipykernel_69560/1700823849.py:41: RuntimeWarning: invalid value encountered in sqrt
  mu_hat = ( nu_cav[i] / tau_cav[i] ) + ( ( y[i] * tau_cav[i]**(-1) * norm.pdf(z_i) ) / ( norm.cdf(z_i) * np.sqrt( 1 + tau_cav[i]**(-1) ) ) )


ValueError: array must not contain infs or NaNs